In [ ]:
import cv2
cv2.setNumThreads(1)
import time
import threading
from dataclasses import dataclass
from typing import Optional, Dict, Tuple
import numpy as np
from collections import Counter

import roboidai as ai
from pygrabber.dshow_graph import FilterGraph
from pyfirmata import ArduinoMega

# ===== 전역 제어 플래그 =====
DNN_LOCK = threading.Lock()
STOP_EVENT = threading.Event()  # q/ESC 누르면 True

# ==============================
# 0. 전역 설정/상수
# ==============================

CAM1_INDEX = 0  # 세트1
CAM2_INDEX = 1  # 세트2
CAM3_INDEX = 2  # 세트3

FRAME_WIDTH = 640
FRAME_HEIGHT = 480
DETECT_INTERVAL = 0.2

CAR_LABELS = ["자동차", "트럭", "버스", "택시", "승합차", "밴", "화물차", "SUV", "픽업트럭"]
PERSON_LABEL = "사람"

S1_SPLIT = 0.865  # S1(세트1) 좌/직 분리 임계 (cx_norm)

# S2, S3: 좌/우 분리 ROI 사용(하단 영역). ROI 밖은 '직진'으로 간주하여 로그 집계
USE_ROI_FOR_S2 = True
USE_ROI_FOR_S3 = True
S2_SPLIT = 0.58  # ROI 미사용시 분리 임계

# 이벤트 조건
VEH_MIN_DURATION = 3.0  # 차량 지속 감지 시간(초)
S1_LEFT_MIN_CNT = 2
S1_STRAIGHT_MIN_CNT = 2
S2_LEFT_MIN_CNT = 2
S3_ANY_MIN_CNT = 3

# 보행자 이벤트
USE_PEDEVENT = True
USE_LINKED_IN_PED_EVENTS = True # 보행자 이벤트 시 연결 차량 신호 동시 점등 여부
PED_MIN_DURATION = 3.0   # 보행자 지속 감지 시간
S1_PED_MIN_CNT = 1
S2_PED_MIN_CNT = 1
S3_PED_MIN_CNT = 1

GRACE_GREEN_SEC = 2.0
GRACE_YELLOW_SEC = 2.0
COOLDOWN_SEC = 5.0 # 이벤트 후 재트리거 쿨다운

# 정상 신호 시간(초)
PHASE_GREEN_SEC = 6.0
PHASE_YELLOW_SEC = 2.0
ALL_RED_BUFFER_SEC = 0.5
PED_WALK_SEC = 6.0

ARDUINO_PORT = 'COM5'

# ==============================
# 1. 유틸/데이터 구조
# ==============================

def list_devices():
    try:
        devs = FilterGraph().get_input_devices()
        print("연결된 카메라 목록:")
        for i, name in enumerate(devs):
            print(f"  Index {i}: {name}")
    except Exception as e:
        print(f"[경고] 장치 조회 실패: {e}")

def open_camera(index: int):
    cap = cv2.VideoCapture(index, cv2.CAP_DSHOW)
    try:
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
    except Exception:
        pass
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
    if not cap.isOpened():
        raise RuntimeError(f"카메라(Index {index})를 열 수 없습니다.")
    print(f"[카메라{index}] 해상도: {int(cap.get(3))}x{int(cap.get(4))}")
    return cap

def setup_detector():
    det = ai.ObjectDetector(multi=True, lang='ko')
    det.download_model()
    det.load_model()
    return det

def normalize_box(x, y, w, h, fw, fh):
    as_x2y2 = (w > fw or h > fh or (x + w/2) > fw or (y + h/2) > fh)
    if as_x2y2:
        x1, y1, x2, y2 = x, y, w, h
        bw = max(0, x2 - x1); bh = max(0, y2 - y1)
        cx = x1 + bw/2.0; cy = y1 + bh/2.0
    else:
        x1, y1 = x, y
        bw, bh = w, h
        x2, y2 = x1 + bw, y1 + bh
        cx = x1 + bw/2.0; cy = y1 + bh/2.0
    return int(x1), int(y1), int(x2), int(y2), int(bw), int(bh), float(cx), float(cy)

def point_in_poly(px, py, poly):
    inside = False
    n = len(poly)
    for i in range(n):
        x1, y1 = poly[i]; x2, y2 = poly[(i+1) % n]
        if ((y1 > py) != (y2 > py)) and (px < (x2-x1)*(py-y1)/(y2-y1 + 1e-9) + x1):
            inside = not inside
    return inside

def left_right_rois(fw, fh):
    """하단 55% 영역을 좌/우 다각형으로 분할 (S2/S3 좌/우/직 로그 집계에 사용)."""
    y_top = 0.55
    left_norm  = [(0.22,y_top),(0.58,y_top),(0.58,0.99),(0.22,0.99)]
    right_norm = [(0.58,y_top),(0.92,y_top),(0.92,0.99),(0.58,0.99)]
    return [(int(x*fw), int(y*fh)) for x,y in left_norm], [(int(x*fw), int(y*fh)) for x,y in right_norm]

def veh_lr_straight_by_roi(cx, cy, fw, fh, use_roi=True, split=0.5) -> str:
    """ROI가 있으면 좌/우, 아니면 cx_norm으로 좌/우, 나머지는 직진."""
    if use_roi:
        lpoly, rpoly = left_right_rois(fw, fh)
        if point_in_poly(cx, cy, lpoly): return "LEFT_TURN"
        if point_in_poly(cx, cy, rpoly): return "RIGHT_TURN"
        return "STRAIGHT"
    else:
        cxn = cx / fw
        if cxn < min(split, 0.45): return "LEFT_TURN"
        elif cxn > max(split, 0.55): return "RIGHT_TURN"
        else: return "STRAIGHT"

@dataclass
class ConditionTimer:
    min_count: int
    min_duration: float
    streak: float = 0.0
    last_met: float = -1e9  # 마지막 조건 충족 완료 timestamp(쿨다운용)

    def update(self, count: int, dt: float):
        if count >= self.min_count:
            self.streak += dt
        else:
            self.streak = 0.0

    def ready_without_consuming(self, now: float, cooldown: float) -> bool:
        """지속시간/쿨다운 충족 여부 확인(소비 X)."""
        return (self.streak >= self.min_duration) and (now - self.last_met) >= cooldown

    def consume(self, now: float):
        """이벤트 발생 소비(쿨다운 시작)."""
        self.last_met = now

class SharedState:
    def __init__(self):
        self.lock = threading.Lock()

        # 현재 프레임 카운트
        self.s1_left = self.s1_straight = self.s1_ped = 0
        self.s2_left = self.s2_any = self.s2_ped = 0  # ★ S2 전체 차량 카운트 포함
        self.s3_any = self.s3_ped = 0

        # 타이머
        self.t_s1_left = ConditionTimer(S1_LEFT_MIN_CNT, VEH_MIN_DURATION)
        self.t_s1_straight = ConditionTimer(S1_STRAIGHT_MIN_CNT, VEH_MIN_DURATION)
        self.t_s2_left = ConditionTimer(S2_LEFT_MIN_CNT, VEH_MIN_DURATION)
        self.t_s3_any = ConditionTimer(S3_ANY_MIN_CNT, VEH_MIN_DURATION)

        # (S1 직진 + S3 전체 차량) ≥ 2, 3초
        self.t_s1s3_straight_combo = ConditionTimer(2, VEH_MIN_DURATION)
        self.last_combo = time.time()

        self.t_s1_ped = ConditionTimer(S1_PED_MIN_CNT, PED_MIN_DURATION)
        self.t_s2_ped = ConditionTimer(S2_PED_MIN_CNT, PED_MIN_DURATION)
        self.t_s3_ped = ConditionTimer(S3_PED_MIN_CNT, PED_MIN_DURATION)

        self.vis_frames = {'S1': None, 'S2': None, 'S3': None}
        self.last_s1 = self.last_s2 = self.last_s3 = time.time() # 마지막 업데이트 시각(세트별)
        self.event_active = False # 이벤트 상태(진행 중 여부)

    # ===== 카운트 업데이트 =====
    def update_s1(self, left_cnt, straight_cnt, ped_cnt):
        now = time.time(); dt = now - self.last_s1; self.last_s1 = now
        with self.lock:
            self.s1_left, self.s1_straight, self.s1_ped = left_cnt, straight_cnt, ped_cnt
            self.t_s1_left.update(left_cnt, dt)
            self.t_s1_straight.update(straight_cnt, dt)
            self.t_s1_ped.update(ped_cnt, dt)

    def update_s2(self, left_cnt, any_cnt, ped_cnt):
        now = time.time(); dt = now - self.last_s2; self.last_s2 = now
        with self.lock:
            self.s2_left, self.s2_any, self.s2_ped = left_cnt, any_cnt, ped_cnt
            self.t_s2_left.update(left_cnt, dt)
            self.t_s2_ped.update(ped_cnt, dt)

    def update_s3(self, any_cnt, ped_cnt):
        now = time.time(); dt = now - self.last_s3; self.last_s3 = now
        with self.lock:
            self.s3_any, self.s3_ped = any_cnt, ped_cnt
            self.t_s3_any.update(any_cnt, dt)
            self.t_s3_ped.update(ped_cnt, dt)

    # ===== 표시 프레임 업데이트/조회 =====
    def set_vis(self, role, frame):
        with self.lock:
            self.vis_frames[role] = frame

    def get_vis_all(self):
        with self.lock:
            return dict(self.vis_frames)

    # ===== 조합 타이머 갱신 (컨트롤러 루프에서 주기적으로 호출) =====
    def update_combo_timer(self):
        now = time.time()
        with self.lock:
            dt = now - self.last_combo; self.last_combo = now
            combo_cnt = self.s1_straight + self.s3_any
            self.t_s1s3_straight_combo.update(combo_cnt, dt)

    # ===== 이벤트 판정/상태 =====
    def phase_satisfies_event(self, current_phase, ev_code) -> bool:
        mapping = {
            'EV_S1_LEFT': {'S1_LEFT'},
            'EV_S1_STRAIGHT': {'S1_STRAIGHT'},
            'EV_S2_LEFT': {'S2_LEFT'},
            'EV_S3_STRAIGHT': {'S1_STRAIGHT'},
            'EV_S1_PED': {'S2_LEFT'},
            'EV_S2_PED': {'S1_STRAIGHT'},
            'EV_S3_PED': {'S1_LEFT'},
        }
        return current_phase in mapping.get(ev_code, set())

        def pick_pending_event(self, current_phase):
        if self.event_active or STOP_EVENT.is_set(): 
            return None

        now = time.time()
        with self.lock:
            # 전체 감지 객체 수 (S1/S2/S3 차량 + 보행자 포함)
            total_objects = (
                self.s1_left + self.s1_straight +
                self.s2_left + self.s2_any +
                self.s3_any +
                self.s1_ped + self.s2_ped + self.s3_ped
            )

            # =======================================
            # 1️. 보행자 이벤트 (인원 수 기반 + 즉시 실행 조건)
            # =======================================
            if USE_PEDEVENT:
                ped_events = [
                    ('EV_S1_PED', self.t_s1_ped, self.s1_ped),
                    ('EV_S2_PED', self.t_s2_ped, self.s2_ped),
                    ('EV_S3_PED', self.t_s3_ped, self.s3_ped),
                ]
                # 감지 인원 많은 순 정렬 (동률 시 기본 순서 유지)
                ped_events.sort(key=lambda x: x[2], reverse=True)

                for ev, t, cnt in ped_events:
                    if self.phase_satisfies_event(current_phase, ev):
                        continue

                    # 주변 객체 1개 이하 & 보행자 ≥ 1명 → 즉시 이벤트 발생
                    if cnt >= 1 and total_objects <= 1:
                        print(f"[우선실행] {ev} : 주변 객체 없음 → 즉시 이벤트 발생")
                        t.consume(now)
                        return ev

                    # 일반 조건 충족 시
                    if t.ready_without_consuming(now, COOLDOWN_SEC):
                        t.consume(now)
                        return ev

            # =======================================
            # 2️. 차량 이벤트 (차량 수 기반 + 즉시 실행 조건)
            # =======================================
            veh_events = [
                ('EV_S1_LEFT', self.t_s1_left, self.s1_left),
                ('EV_S1_STRAIGHT', self.t_s1s3_straight_combo, self.s1_straight + self.s3_any),
                ('EV_S2_LEFT', self.t_s2_left, self.s2_left),
            ]
            # 차량 수 많은 순 정렬 (동률 시 기존 순서 유지)
            veh_events.sort(key=lambda x: x[2], reverse=True)

            for ev, t, cnt in veh_events:
                if self.phase_satisfies_event(current_phase, ev):
                    continue

                # 주변 객체 1개 이하 & 해당 방향 차량 ≥ 1대 → 즉시 이벤트 발생
                if cnt >= 1 and total_objects <= 1:
                    print(f"[우선실행] {ev} : 주변 객체 없음 → 즉시 이벤트 발생")
                    t.consume(now)
                    return ev

                # 일반 조건 충족 시
                if t.ready_without_consuming(now, COOLDOWN_SEC):
                    t.consume(now)
                    return ev

        return None


    def set_event_active(self, active): self.event_active = active

# ==============================
# 2. 아두이노 I/O (핀과 단계 함수)
# ==============================

board = ArduinoMega(ARDUINO_PORT)

# 핀 정의
car1_green, car1_yellow, car1_red = 22, 23, 24
ped1_green, ped1_red = 25, 26
car2_green, car2_yellow, car2_red = 27, 28, 29
ped2_green, ped2_red = 30, 31
car3_green, car3_yellow, car3_red = 32, 33, 34
ped3_green, ped3_red = 35, 36

all_pins = [
    car1_green, car1_yellow, car1_red, ped1_green, ped1_red,
    car2_green, car2_yellow, car2_red, ped2_green, ped2_red,
    car3_green, car3_yellow, car3_red, ped3_green, ped3_red
]
for p in all_pins:
    board.digital[p].mode = 1

def set_traffic1(car_g, car_y, car_r, ped_g, ped_r):
    board.digital[car1_green].write(car_g)
    board.digital[car1_yellow].write(car_y)
    board.digital[car1_red].write(car_r)
    board.digital[ped1_green].write(ped_g)
    board.digital[ped1_red].write(ped_r)

def set_traffic2(car_g, car_y, car_r, ped_g, ped_r):
    board.digital[car2_green].write(car_g)
    board.digital[car2_yellow].write(car_y)
    board.digital[car2_red].write(car_r)
    board.digital[ped2_green].write(ped_g)
    board.digital[ped2_red].write(ped_r)

def set_traffic3(car_g, car_y, car_r, ped_g, ped_r):
    board.digital[car3_green].write(car_g)
    board.digital[car3_yellow].write(car_y)
    board.digital[car3_red].write(car_r)
    board.digital[ped3_green].write(ped_g)
    board.digital[ped3_red].write(ped_r)

def all_red():
    set_traffic1(0,0,1,0,1)
    set_traffic2(0,0,1,0,1)
    set_traffic3(0,0,1,0,1)

def sleep_until(sec: float):
    t0 = time.time()
    while not STOP_EVENT.is_set() and (time.time() - t0 < sec):
        time.sleep(0.02)

# ---- 정상 단계 (기본 사이클, 연결 유지) ----
def phase_s1_left(duration: float):
    # S1 좌회전 + S3 보행자
    set_traffic1(1,0,1,0,1)
    set_traffic2(0,0,1,0,1)
    set_traffic3(0,0,1,1,0)
    sleep_until(duration)

def phase_s1_left_yellow(duration: float = PHASE_YELLOW_SEC):
    set_traffic1(0,1,0,0,1)
    set_traffic2(0,0,1,0,1)
    set_traffic3(0,0,1,0,1)
    sleep_until(duration)

def phase_s1_straight(duration: float):
    # S1 직진 + S3 직진 + S2 보행자
    set_traffic1(1,0,0,0,1)
    set_traffic2(0,0,1,1,0)
    set_traffic3(1,0,0,0,1)
    sleep_until(duration)

def phase_s1_straight_yellow(duration: float = PHASE_YELLOW_SEC):
    set_traffic1(0,1,0,0,1)
    set_traffic2(0,0,1,0,1)
    set_traffic3(0,1,0,0,1)
    sleep_until(duration)

def phase_s2_left(duration: float):
    # S2 좌회전 + S1 보행자  (차량 초록+빨강 동시 점등)
    set_traffic1(0,0,1,1,0)
    set_traffic2(1,0,1,0,1)
    set_traffic3(0,0,1,0,1)
    sleep_until(duration)

def phase_s2_left_yellow(duration: float = PHASE_YELLOW_SEC):
    set_traffic1(0,0,1,0,1)
    set_traffic2(0,1,0,0,1)
    set_traffic3(0,0,1,0,1)
    sleep_until(duration)

def phase_all_red_buffer(duration: float = ALL_RED_BUFFER_SEC):
    all_red()
    sleep_until(duration)

# ---- 이벤트(연결형) 단계 ----
def ev_go_s1_left(duration: float):            phase_s1_left(duration)
def ev_yellow_s1_left(duration: float = PHASE_YELLOW_SEC):   phase_s1_left_yellow(duration)
def ev_go_s1_straight(duration: float):       phase_s1_straight(duration)
def ev_yellow_s1_straight(duration: float = PHASE_YELLOW_SEC): phase_s1_straight_yellow(duration)
def ev_go_s2_left(duration: float):           phase_s2_left(duration)
def ev_yellow_s2_left(duration: float = PHASE_YELLOW_SEC):   phase_s2_left_yellow(duration)
def ev_go_s3_straight(duration: float):       phase_s1_straight(duration)       # 연결 조합 동일
def ev_yellow_s3_straight(duration: float = PHASE_YELLOW_SEC): phase_s1_straight_yellow(duration)

# 보행자 이벤트
def ev_go_s1_ped(duration: float):
    if USE_LINKED_IN_PED_EVENTS:
        # S1 보행 + S2 좌회전(연결)
        set_traffic1(0,0,1,1,0)
        set_traffic2(1,0,1,0,1)
        set_traffic3(0,0,1,0,1)
    else:
        set_traffic1(0,0,1,1,0); set_traffic2(0,0,1,0,1); set_traffic3(0,0,1,0,1)
    sleep_until(duration)

def ev_clear_s1_ped():
    if USE_LINKED_IN_PED_EVENTS:
        phase_s2_left_yellow(PHASE_YELLOW_SEC)
    phase_all_red_buffer(ALL_RED_BUFFER_SEC)

def ev_go_s2_ped(duration: float):
    if USE_LINKED_IN_PED_EVENTS:
        # S2 보행 + S1 직진 + S3 직진(연결)
        set_traffic1(1,0,0,0,1)
        set_traffic2(0,0,1,1,0)
        set_traffic3(1,0,0,0,1)
    else:
        set_traffic1(0,0,1,0,1); set_traffic2(0,0,1,1,0); set_traffic3(0,0,1,0,1)
    sleep_until(duration)

def ev_clear_s2_ped():
    if USE_LINKED_IN_PED_EVENTS:
        phase_s1_straight_yellow(PHASE_YELLOW_SEC)
    phase_all_red_buffer(ALL_RED_BUFFER_SEC)

def ev_go_s3_ped(duration: float):
    if USE_LINKED_IN_PED_EVENTS:
        # S3 보행 + S1 좌회전(연결)
        set_traffic1(1,0,1,0,1)
        set_traffic2(0,0,1,0,1)
        set_traffic3(0,0,1,1,0)
    else:
        set_traffic1(0,0,1,0,1); set_traffic2(0,0,1,0,1); set_traffic3(0,0,1,1,0)
    sleep_until(duration)

def ev_clear_s3_ped():
    if USE_LINKED_IN_PED_EVENTS:
        phase_s1_left_yellow(PHASE_YELLOW_SEC)
    phase_all_red_buffer(ALL_RED_BUFFER_SEC)

# ==============================
# 3. 카메라 스레드
# ==============================

def _format_counts(cam_name: str, role: str, label_counts: Counter,
                   veh_L: int, veh_S: int, veh_R: int):
    # 라벨 카운트 요약(사람, 차량계, 기타 top3)
    people = label_counts.get(PERSON_LABEL, 0)
    veh_total = sum(label_counts[l] for l in CAR_LABELS)
    # 기타 상위 3개 (사람/차량 제외)
    others = {k: v for k, v in label_counts.items() if k != PERSON_LABEL and k not in CAR_LABELS}
    others_top = ", ".join([f"{k}={v}" for k, v in sorted(others.items(), key=lambda x: -x[1])[:3]]) if others else "-"
    print(f"[{cam_name}/{role}] 라벨: 차량계={veh_total}, 사람={people}, 기타={others_top} | 차량 분류 L={veh_L}, S={veh_S}, R={veh_R}", flush=True)

class CameraWorker(threading.Thread):
    def __init__(self, cam_index: int, set_role: str, shared: SharedState, cam_name: str):
        """
        set_role:
          'S1' : 세트1 (좌/직 분리 by split)
          'S2' : 세트2 (좌/우/직 ROI로 분류; 이벤트용으로 '좌회전'과 '전체 차량' 둘 다 집계)
          'S3' : 세트3 (좌/우/직 ROI로 분류; 이벤트는 ANY로만 사용)
        """
        super().__init__(daemon=True)
        self.index = cam_index
        self.role = set_role  # 'S1' / 'S2' / 'S3'
        self.cam_name = cam_name
        self.shared = shared
        self.cap = open_camera(cam_index)
        self.det = setup_detector()
        self.last_detect_t = 0.0

    def run(self):
        while not STOP_EVENT.is_set():
            ok, frame = self.cap.read()
            if not ok:
                time.sleep(0.01)
                continue

            now = time.time()
            do_detect = (now - self.last_detect_t) >= DETECT_INTERVAL
            labels, boxes = [], []

            if do_detect:
                self.last_detect_t = now
                try:
                    # --- DNN 호출을 전역 락으로 직렬화 + GPU 비활성 ---
                    with DNN_LOCK:
                        ok_det = self.det.detect(frame, gpu=False)
                    if ok_det:
                        labels = self.det.get_label()
                        boxes  = self.det.get_box()
                except cv2.error:
                    labels, boxes = [], []

                # 집계 및 로그
                try:
                    label_counts, vehL, vehS, vehR = self._aggregate_and_log(frame, labels, boxes)
                    _format_counts(self.cam_name, self.role, label_counts, vehL, vehS, vehR)
                except Exception:
                    pass

            # 표시 프레임 업데이트(바운딩박스+라벨만)
            try:
                out = self.det.draw_result(frame) if labels or boxes else frame
            except Exception:
                out = frame
            self.shared.set_vis(self.role, out)

        self.cap.release()

    # 공통: 집계 + 공유상태 업데이트 + 차량 좌/직/우 로그 분류
    def _aggregate_and_log(self, frame, labels, boxes) -> Tuple[Counter, int, int, int]:
        fh, fw = frame.shape[:2]
        label_counts = Counter(labels)

        if self.role == 'S1':
            left_cnt = 0; straight_cnt = 0; ped_cnt = label_counts.get(PERSON_LABEL, 0)
            for lab, (x, y, w, h) in zip(labels, boxes):
                if lab in CAR_LABELS:
                    x1,y1,x2,y2,bw,bh,cx,cy = normalize_box(x,y,w,h, fw, fh)
                    cx_norm = cx / fw
                    if cx_norm < S1_SPLIT: left_cnt += 1
                    else:                   straight_cnt += 1
            # 공유 상태 업데이트(이벤트용)
            self.shared.update_s1(left_cnt, straight_cnt, ped_cnt)
            # 로그용 R=0
            return label_counts, left_cnt, straight_cnt, 0

        elif self.role == 'S2':
            left_cnt = 0; right_cnt = 0; straight_cnt = 0
            any_cnt = 0
            ped_cnt = label_counts.get(PERSON_LABEL, 0)
            for lab, (x, y, w, h) in zip(labels, boxes):
                if lab in CAR_LABELS:
                    any_cnt += 1
                    x1,y1,x2,y2,bw,bh,cx,cy = normalize_box(x,y,w,h, fw, fh)
                    cls = veh_lr_straight_by_roi(cx, cy, fw, fh, use_roi=USE_ROI_FOR_S2, split=S2_SPLIT)
                    if cls == "LEFT_TURN":      left_cnt += 1
                    elif cls == "RIGHT_TURN":   right_cnt += 1
                    else:                       straight_cnt += 1
            # 이벤트: 좌회전 + 전체 차량 둘 다 갱신
            self.shared.update_s2(left_cnt, any_cnt, ped_cnt)
            return label_counts, left_cnt, straight_cnt, right_cnt

        else:  # self.role == 'S3'
            left_cnt = 0; right_cnt = 0; straight_cnt = 0
            ped_cnt = label_counts.get(PERSON_LABEL, 0)
            any_cnt = 0
            for lab, (x, y, w, h) in zip(labels, boxes):
                if lab in CAR_LABELS:
                    any_cnt += 1
                    x1,y1,x2,y2,bw,bh,cx,cy = normalize_box(x,y,w,h, fw, fh)
                    cls = veh_lr_straight_by_roi(cx, cy, fw, fh, use_roi=USE_ROI_FOR_S3, split=0.5)
                    if cls == "LEFT_TURN":      left_cnt += 1
                    elif cls == "RIGHT_TURN":   right_cnt += 1
                    else:                       straight_cnt += 1
            # 이벤트용: ANY 차량 수 사용
            self.shared.update_s3(any_cnt, ped_cnt)
            return label_counts, left_cnt, straight_cnt, right_cnt

# ==============================
# 4. 디스플레이(UI) 스레드
# ==============================

class DisplayWorker(threading.Thread):
    def __init__(self, shared: SharedState):
        super().__init__(daemon=True)
        self.shared = shared
        self.win_names = {'S1': f"Cam{CAM1_INDEX} (Set1)",
                          'S2': f"Cam{CAM2_INDEX} (Set2)",
                          'S3': f"Cam{CAM3_INDEX} (Set3)"}

    def run(self):
        # 윈도우 생성은 UI 스레드에서
        for name in self.win_names.values():
            cv2.namedWindow(name, cv2.WINDOW_NORMAL)
            cv2.resizeWindow(name, FRAME_WIDTH, FRAME_HEIGHT)  # 창 크기 고정 640x480

        while not STOP_EVENT.is_set():
            frames = self.shared.get_vis_all()
            for role, name in self.win_names.items():
                f = frames.get(role)
                if f is None:
                    f = np.zeros((FRAME_HEIGHT, FRAME_WIDTH, 3), dtype=np.uint8)
                # 혹시 프레임 크기가 다르면 표시용으로 리사이즈
                if f.shape[1] != FRAME_WIDTH or f.shape[0] != FRAME_HEIGHT:
                    f = cv2.resize(f, (FRAME_WIDTH, FRAME_HEIGHT), interpolation=cv2.INTER_NEAREST)
                cv2.imshow(name, f)

            key = cv2.waitKey(1) & 0xFF
            if key in (ord('q'), 27):  # q or ESC
                STOP_EVENT.set()
                break

        cv2.destroyAllWindows()

# ==============================
# 5. 컨트롤러(FSM + 연결형 이벤트)
# ==============================

class Controller:
    """
    기본 사이클:
      S1_LEFT → YELLOW → S1_STRAIGHT → YELLOW → S2_LEFT → YELLOW → ALL_RED
    이벤트: 유예(2s green + 2s yellow) → 연결형 이벤트 본편 → (필요 시 yellow) → all-red → 정상 '다음 단계' 복귀
    """
    def __init__(self, shared: SharedState):
        self.shared = shared
        self.current_phase = 'S1_LEFT'
        self.cycle = [
            ('S1_LEFT', phase_s1_left, phase_s1_left_yellow),
            ('S1_STRAIGHT', phase_s1_straight, phase_s1_straight_yellow),
            ('S2_LEFT', phase_s2_left, phase_s2_left_yellow),
        ]
        self.cycle_idx = 0

    def run(self):
        while not STOP_EVENT.is_set():
            # ★ 조합 타이머를 주기적으로 갱신
            self.shared.update_combo_timer()

            name, go_fn, yellow_fn = self.cycle[self.cycle_idx]
            self.current_phase = name

            # 이벤트 체크
            ev = self.shared.pick_pending_event(self.current_phase)
            if ev:
                self._run_event_with_grace(current_name=name, current_go=go_fn, current_yellow=yellow_fn, event_name=ev)
                # 이벤트 후 정상 '다음 단계'부터
                self.cycle_idx = (self.cycle_idx + 1) % len(self.cycle)
                continue

            # 정상 실행
            go_fn(PHASE_GREEN_SEC)
            yellow_fn(PHASE_YELLOW_SEC)
            phase_all_red_buffer(ALL_RED_BUFFER_SEC)

            # 다음 단계
            self.cycle_idx = (self.cycle_idx + 1) % len(self.cycle)

    def _run_event_with_grace(self, current_name: str, current_go, current_yellow, event_name: str):
        print(f"[이벤트] {event_name} 요청 → 유예 진행")
        self.shared.set_event_active(True)

        # 1) 현재 그린 2초 유지
        current_go(GRACE_GREEN_SEC)
        # 2) 현재 옐로 2초
        current_yellow(GRACE_YELLOW_SEC)
        phase_all_red_buffer(ALL_RED_BUFFER_SEC)

        # 3) 연결형 이벤트 본편
        if event_name == 'EV_S1_LEFT':
            ev_go_s1_left(PHASE_GREEN_SEC)
            ev_yellow_s1_left(PHASE_YELLOW_SEC)
        elif event_name == 'EV_S1_STRAIGHT':
            ev_go_s1_straight(PHASE_GREEN_SEC)
            ev_yellow_s1_straight(PHASE_YELLOW_SEC)
        elif event_name == 'EV_S2_LEFT':
            ev_go_s2_left(PHASE_GREEN_SEC)
            ev_yellow_s2_left(PHASE_YELLOW_SEC)
        elif event_name == 'EV_S3_STRAIGHT':
            ev_go_s3_straight(PHASE_GREEN_SEC)
            ev_yellow_s3_straight(PHASE_YELLOW_SEC)
        elif event_name == 'EV_S1_PED':
            ev_go_s1_ped(PED_WALK_SEC)
            ev_clear_s1_ped()
        elif event_name == 'EV_S2_PED':
            ev_go_s2_ped(PED_WALK_SEC)
            ev_clear_s2_ped()
        elif event_name == 'EV_S3_PED':
            ev_go_s3_ped(PED_WALK_SEC)
            ev_clear_s3_ped()

        # 4) all-red buffer 후 종료
        phase_all_red_buffer(ALL_RED_BUFFER_SEC)
        self.shared.set_event_active(False)
        print(f"[이벤트] {event_name} 완료 → 정상 다음 단계로 복귀")

# ==============================
# 6. 메인
# ==============================

def main():
    list_devices()

    shared = SharedState()

    # Cam0->세트1, Cam1->세트2, Cam2->세트3
    cam1 = CameraWorker(CAM1_INDEX, 'S1', shared, cam_name=f"Cam{CAM1_INDEX}")
    cam2 = CameraWorker(CAM2_INDEX, 'S2', shared, cam_name=f"Cam{CAM2_INDEX}")
    cam3 = CameraWorker(CAM3_INDEX, 'S3', shared, cam_name=f"Cam{CAM3_INDEX}")

    ui = DisplayWorker(shared)
    ctrl = Controller(shared)

    try:
        cam1.start(); cam2.start(); cam3.start()
        ui.start()
        ctrl.run()
    except KeyboardInterrupt:
        STOP_EVENT.set()
    finally:
        STOP_EVENT.set()
        cam1.join(timeout=2); cam2.join(timeout=2); cam3.join(timeout=2)
        ui.join(timeout=2)

        for p in all_pins:
            board.digital[p].write(0)
        board.exit()
        cv2.destroyAllWindows()
        print("종료 및 모든 신호 OFF")

if __name__ == "__main__":
    main()